In [134]:
import numpy as np
from scipy.sparse import coo_matrix
from qiskit_aer import AerSimulator
from qiskit.quantum_info import Statevector

from QLS.numpy_linear_solver import NumPyLinearSolver   # classical
from QLS.hhl import HHL              # quantum HHL
import time

In [135]:
# ----- user parameters ------------------------------------------------------
make_hermitian = True       # True → Hermitian SPD; False → general non-Hermitian
target_kappa   = 1e3        # desired condition number κ(A)
density        = 0.9        # fraction of nonzero entries (0<density≤1)
noise_level    = 1e1       # relative off-diag noise for SPD case

In [136]:
# ----- problem definition ---------------------------------------------------
NUM_WORK_QUBITS = 3                
DIM             = 2 ** NUM_WORK_QUBITS

In [137]:
# ----- helpers ---------------------------------------------------------------
def generate_sparse_spd(n, kappa, density, noise_level):
    """Hermitian SPD with log-spaced eigenvalues and random sparse off-diag noise."""
    # 1) log-spaced eigenvalues
    eigs = np.logspace(0, np.log10(kappa), n)
    # 2) diagonal entries
    rows = np.arange(n); cols = rows; data = eigs
    A = coo_matrix((data, (rows, cols)), shape=(n, n))
    # 3) add symmetric off-diagonal noise
    total = n*n
    nnz   = int(density*total)
    off   = max(nnz - n, 0)
    off  -= off % 2
    half  = off//2
    if half>0:
        i = np.random.randint(0,n,half*2)
        j = np.random.randint(0,n,half*2)
        mask = (i!=j)
        i,j = i[mask][:half], j[mask][:half]
        eps  = noise_level * eigs.min()
        vals = np.random.uniform(-eps, eps, size=half)
        A   = A + coo_matrix((vals,(i,j)),shape=(n,n))\
                + coo_matrix((vals,(j,i)),shape=(n,n))
    return A.tocsr()

In [138]:
def generate_general(n, kappa, density):
    """General (non-Hermitian) matrix with approx κ via SVD, then sparsified."""
    # 1) U, V random orthonormal
    U,_  = np.linalg.qr(np.random.randn(n,n))
    V,_  = np.linalg.qr(np.random.randn(n,n))
    # 2) singular values
    s    = np.logspace(0, np.log10(kappa), n)
    A    = U @ np.diag(s) @ V.T
    # 3) sparsify by zeroing random entries
    mask = (np.random.rand(n,n) < density)
    A   *= mask
    return A

In [139]:
# ----- build A ----------------------------------------------------------------
if make_hermitian:
    A_sparse = generate_sparse_spd(DIM, target_kappa, density, noise_level)
    A        = A_sparse.toarray()
else:
    A = generate_general(DIM, target_kappa, density)


In [140]:
# ----- checks & print ---------------------------------------------------------
is_herm = np.allclose(A, A.conj().T, atol=1e-12)
cond_A  = np.linalg.cond(A)
nnz     = np.count_nonzero(A)
print(f"A (dim={DIM}×{DIM}), Hermitian? {is_herm}, κ(A) ≈ {cond_A:.3e}, "
      f"sparsity={nnz}/{DIM*DIM}={nnz/(DIM*DIM):.2%}\n")
print("A =\n", A)   # uncomment to see the full matrix

A (dim=8×8), Hermitian? True, κ(A) ≈ 2.725e+02, sparsity=44/64=68.75%

A =
 [[ 1.00000000e+00  4.24800989e+00  0.00000000e+00 -7.74777271e-01
   1.70555570e+00  2.73639687e+00 -5.73435844e+00 -1.40243447e+01]
 [ 4.24800989e+00  2.68269580e+00  0.00000000e+00  0.00000000e+00
  -7.26352256e+00  1.92791068e+00 -7.25368270e+00  1.27688457e+01]
 [ 0.00000000e+00  0.00000000e+00  7.19685673e+00  0.00000000e+00
   5.29560571e+00  0.00000000e+00  8.26232856e+00  0.00000000e+00]
 [-7.74777271e-01  0.00000000e+00  0.00000000e+00  1.93069773e+01
   9.77360325e+00 -2.69167383e+00  0.00000000e+00  3.42730881e+00]
 [ 1.70555570e+00 -7.26352256e+00  5.29560571e+00  9.77360325e+00
   5.17947468e+01  0.00000000e+00  9.75739249e+00  0.00000000e+00]
 [ 2.73639687e+00  1.92791068e+00  0.00000000e+00 -2.69167383e+00
   0.00000000e+00  1.38949549e+02  0.00000000e+00  1.80469990e+00]
 [-5.73435844e+00 -7.25368270e+00  8.26232856e+00  0.00000000e+00
   9.75739249e+00  0.00000000e+00  3.72759372e+02  8.3790752

In [141]:
# ----- right-hand side -------------------------------------------------------
b_vec = np.zeros(DIM, dtype=complex if np.iscomplexobj(A) else float)
b_vec[0] = 1
b_vec

array([1., 0., 0., 0., 0., 0., 0., 0.])

In [142]:
# ----- classical solution ----------------------------------------------------
classical_res = NumPyLinearSolver().solve(
    A,
    b_vec / np.linalg.norm(b_vec)
)

In [143]:
# ----- quantum (HHL) solution ------------------------------------------------
# calculate the time for the quantum solver
start_time = time.time()
backend     = AerSimulator()
hhl_solver  = HHL(epsilon=1e-3, quantum_instance=backend)
quantum_res = hhl_solver.solve(A, b_vec)

def extract_solution(result, n_work_qubits: int) -> np.ndarray:
    sv           = Statevector(result.state).data
    total_qubits = int(np.log2(len(sv)))
    base_index   = 1 << (total_qubits - 1)
    amps         = np.array([sv[base_index + i]
                             for i in range(2 ** n_work_qubits)])
    return result.euclidean_norm * amps / np.linalg.norm(amps)

x_classical = classical_res.state
x_quantum   = extract_solution(quantum_res, NUM_WORK_QUBITS)
# calculate the time taken by the quantum solver
end_time = time.time()
# print time taken
print(f"Quantum solver took {end_time - start_time:.3f} seconds.")

Quantum solver took 43.752 seconds.


In [144]:
# ----- results ---------------------------------------------------------------
print("Classical solution vector:", x_classical)
print("Quantum   solution vector:", x_quantum, "\n")
print("Classical Euclidean norm:", classical_res.euclidean_norm)
print("Quantum   Euclidean norm:", quantum_res.euclidean_norm, "\n")
print("‖x_classical − x_quantum‖₂ =",
      np.linalg.norm(x_classical - x_quantum))

Classical solution vector: [-0.0548339   0.22242834 -0.03312921 -0.02212439  0.0399474  -0.0023887
  0.00325338 -0.00355628]
Quantum   solution vector: [-0.05779432-9.94745074e-13j  0.21849621+5.25676136e-13j
 -0.03446888-3.22653202e-14j -0.02201319-8.49122535e-14j
  0.03960608+1.16159584e-13j -0.00226944+1.09321857e-14j
  0.00316885-7.07418974e-15j -0.00354747-2.01905550e-14j] 

Classical Euclidean norm: 0.23599341071764052
Quantum   Euclidean norm: 0.23313060464389712 

‖x_classical − x_quantum‖₂ = 0.005115744720575678
